In [1]:
import openai

from fp_dataset_artifacts.utils import init_openai
from datasets import list_datasets, load_dataset, list_metrics, load_metric

init_openai()

data = load_dataset('boolq')
data

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/952 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.53M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset boolq downloaded and prepared to /home/x/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270
    })
})

In [2]:
data['train'][0]

{'question': 'do iran and afghanistan speak the same language',
 'answer': True,
 'passage': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.'}

In [5]:
# Pretend this is for a chat bot and has to deal with no context or structure
def get_bare_prompt(x):
    return f"{x['question']}"
prompt = get_bare_prompt(data['train'][0])
print(prompt)

do iran and afghanistan speak the same language


In [6]:
def get_response(prompt):
    response = openai.Completion.create(
      engine='curie',
      prompt=prompt,
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=["\n"]
    )
    return response

get_response(prompt)

<OpenAIObject text_completion id=cmpl-4BUqUNwKvdUa43XyoWL6r7TMijAdP at 0x7fd51caa0950> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "."
    }
  ],
  "created": 1638645526,
  "id": "cmpl-4BUqUNwKvdUa43XyoWL6r7TMijAdP",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}

In [10]:
# Make it better
intro = 'I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will answer in either "Yes" or "No".'

def clean_question(q):
    return q.capitalize() + '?'

def get_prompt(x):
    return f"{intro}\n\nQ: {clean_question(x['question'])}\n\nA: "
prompt = get_prompt(data['train'][0])
print(prompt)

I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will answer in either "Yes" or "No".

Q: Do iran and afghanistan speak the same language?

A: 


In [11]:
get_response(prompt)

<OpenAIObject text_completion id=cmpl-4BUsuLf7ZUnUs4DXS48MASdf3fgS4 at 0x7fd51c7bb7c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "????"
    }
  ],
  "created": 1638645676,
  "id": "cmpl-4BUsuLf7ZUnUs4DXS48MASdf3fgS4",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}

In [14]:
# With passage

def get_passage_prompt(x):
    return f"{intro}\n\n{x['passage']}\n\nQ: {clean_question(x['question'])}\n\nA: "
prompt = get_passage_prompt(data['train'][0])
print(prompt)

I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will answer in either "Yes" or "No".

Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.

Q: Do iran and afghanistan speak the same language?

A: 


In [15]:
get_response(prompt)

<OpenAIObject text_completion id=cmpl-4BUu0CsKrJKAzPrdcPzgXfCTkwJwK at 0x7fd5783975e0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\u06cc\u06a9\u06cc \u0627\u0632 \u062f\u06cc\u06af\u0631 \u062f\u0648\u0644\u062a \u0647\u0627\u06cc \u062c\u0647\u0627\u0646 \u0628\u0647 \u0639\u0646\u0648\u0627\u0646 \u06cc\u06a9 \u062f\u0648\u0644\u062a \u062f\u0631 \u062c\u0647\u0627\u0646 \u0628\u0647 \u0639\u0646\u0648\u0627\u0646 \u06cc\u06a9 \u062f\u0648\u0644\u062a \u062f\u0631 \u062c\u0647\u0627\u0646 \u0628\u0647 \u0639\u0646\u0648\u0627\u0646 \u06cc\u06a9"
    }
  ],
  "created": 1638645744,
  "id": "cmpl-4BUu0CsKrJKAzPrdcPzgXfCTkwJwK",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}

In [16]:
# Different example
prompt = get_passage_prompt(data['train'].shuffle()[0])
print(prompt)

I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will answer in either "Yes" or "No".

Harry Potter and the Forbidden Journey uses KUKA robocoaster technology, which allows the seats to pivot while being held above the track by a robotic arm. However, the ride is not a roller coaster but a scenic dark ride. The experience includes a flight around Hogwarts castle, an encounter with the Whomping Willow and a horde of Dementors, and a Quidditch match. The ride drops, spins around, twists and turns, but does not turn upside down, though passengers sometimes lie flat on their backs. Over-the-shoulder bars are used to secure guests in their seats, and a single parabolic metal bar is used as a hand grip. At the conclusion of the ride, guests exit into Filch's Emporium of Confiscated Goods gift shop.

Q: Is the harry potter and the forbidden journey ride a roller coaster?

A: 


In [17]:
get_response(prompt)

<OpenAIObject text_completion id=cmpl-4BUuNjjwOefYrjdNsj5gTNMUzhjHH at 0x7fd51ca3a130> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "ive been on it and it isnt a roller coaster."
    }
  ],
  "created": 1638645767,
  "id": "cmpl-4BUuNjjwOefYrjdNsj5gTNMUzhjHH",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}

In [18]:
# Another one
prompt = get_passage_prompt(data['train'].shuffle()[1])
print(prompt)

I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will answer in either "Yes" or "No".

Goalkeepers are normally allowed to handle the ball within their own penalty area, and once they have control of the ball in their hands opposition players may not challenge them for it. However the back-pass rule prohibits goalkeepers from handling the ball after it has been deliberately kicked to them by a team-mate, or after receiving it directly from a throw-in taken by a team-mate. Back-passes with parts of the body other than the foot, such as headers, are not prohibited. Despite the popular name ``back-pass rule'', there is no requirement in the laws that the kick or throw-in must be backwards; handling by the goalkeeper is forbidden regardless of the direction the ball travels.

Q: Can you pass to the goalie in soccer?

A: 


In [19]:
get_response(prompt)

<OpenAIObject text_completion id=cmpl-4BUumY2sdMVaK8MHxi9YL4ggTjA7S at 0x7fd51c134db0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638645792,
  "id": "cmpl-4BUumY2sdMVaK8MHxi9YL4ggTjA7S",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}

In [21]:
# Try without the stop parameter
def get_no_stop_response(prompt):
    response = openai.Completion.create(
      engine='curie',
      prompt=prompt,
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
    )
    return response

get_no_stop_response(prompt)

<OpenAIObject text_completion id=cmpl-4BV1aP6BXjVlR5tn5K7LQBgJnsMTl at 0x7fd51ca585e0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638646214,
  "id": "cmpl-4BV1aP6BXjVlR5tn5K7LQBgJnsMTl",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}